In [7]:
import os
import json
import openai
from openai import AzureOpenAI

In [8]:
AZURE_OPENAI_API_VERSION='2025-01-01-preview'
AZURE_OPENAI_EMBEDDINGS_API_VERSION='2023-05-15'

In [9]:
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_version=AZURE_OPENAI_API_VERSION
)

In [10]:
CHAT_DEPLOYMENT = "gpt-5-chat"             
EMBEDDING_DEPLOYMENT = "text-embedding-3-small"

In [11]:
def generate_caption_and_tags(vision_summary_json):

    system_prompt = (
        "Jesteś asystentem generującym krótki opis zdjęcia i listę tagów w języku polskim.\n"
        "Weź pod uwagę wykryte etykiety, opis i tekst (OCR) dostarczony poniżej.\n"
        "Opis zdjęcia powinien być utrzymany w dziennikarskim stylu i nadawać się do publikacji (1-2 zdania). Nie może być suchym opisem widoku\n"
        "Zwróć w formacie JSON z kluczami: 'caption' (1-2 zdania), "
        "'tags' (lista obiektów {tag, confidence}).\n"
        "Użyj naturalnego języka polskiego."
    )

    user_prompt = f"Vision JSON: {vision_summary_json}"

    response = client.chat.completions.create(
        model=CHAT_DEPLOYMENT,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        max_tokens=300,
        temperature=0.2,
    )

    text = response.choices[0].message.content

    # Expected JSON output
    try:
        return json.loads(text)
    except Exception:
        return {"raw": text}

In [12]:
sample_json = {'captions': [{'text': 'a group of people holding signs', 'confidence': 0.6494020819664001}], 'tags': [{'name': 'clothing', 'confidence': 0.9988632798194885}, {'name': 'human face', 'confidence': 0.9978727102279663}, {'name': 'person', 'confidence': 0.9968821406364441}, {'name': 'woman', 'confidence': 0.9683936834335327}, {'name': 'text', 'confidence': 0.956762969493866}, {'name': 'man', 'confidence': 0.9565277099609375}, {'name': 'smile', 'confidence': 0.9443103075027466}, {'name': 'newspaper', 'confidence': 0.8428946137428284}, {'name': 'outdoor', 'confidence': 0.8321356773376465}, {'name': 'group', 'confidence': 0.7874609231948853}, {'name': 'sign', 'confidence': 0.7766597867012024}, {'name': 'people', 'confidence': 0.7492111921310425}], 'ocr_text': 'TRUMI\nTRUMP\nWe WeTAFAGAIN!\nMAKE AMERICA GREAT AGAIN\nTRUMP\nthe silent majority\nMEINERICA GREAT AGAIN!\nT\nSTANDS WITH\nTRUMP\nSUME\nNEM\nWent may\nRUMP\nS W\nAMERI GREAT AGAIN!\nTR\nSTAN\nTRUMP\nthe site\nRUI\nARE AMERICA GREAT AGAIN\nSTAN\nWITH\n'}

In [13]:
sample_json2 = {'captions': [{'text': 'a group of jockeys racing horses', 'confidence': 0.5538452863693237}], 'tags': [{'name': 'horse', 'confidence': 0.9770833253860474}, {'name': 'stadium', 'confidence': 0.9752413034439087}, {'name': 'outdoor', 'confidence': 0.9523265957832336}, {'name': 'building', 'confidence': 0.9427610039710999}, {'name': 'rein', 'confidence': 0.9391779899597168}, {'name': 'bridle', 'confidence': 0.9323909282684326}, {'name': 'stallion', 'confidence': 0.9239025712013245}, {'name': 'horse tack', 'confidence': 0.9214749336242676}, {'name': 'horse supplies', 'confidence': 0.9210265874862671}, {'name': 'flat racing', 'confidence': 0.9156216979026794}, {'name': 'halter', 'confidence': 0.9028061628341675}, {'name': 'mare', 'confidence': 0.8987149596214294}, {'name': 'equestrianism', 'confidence': 0.8742722868919373}, {'name': 'saddle', 'confidence': 0.8724848031997681}, {'name': 'horse racing', 'confidence': 0.870876133441925}, {'name': 'jockey', 'confidence': 0.8687554597854614}, {'name': 'animal sports', 'confidence': 0.8578435778617859}, {'name': 'grass', 'confidence': 0.8521130084991455}, {'name': 'mane', 'confidence': 0.8485252857208252}, {'name': 'racing', 'confidence': 0.8210968375205994}, {'name': 'person', 'confidence': 0.6902799606323242}, {'name': 'riding', 'confidence': 0.6800144910812378}, {'name': 'people', 'confidence': 0.5848963856697083}, {'name': 'track', 'confidence': 0.5521560311317444}], 'ocr_text': 'Torei\nHELIOS EXPRESS\nSRISIG\nTONO REF,\nO\n'}

In [14]:
generate_caption_and_tags(sample_json)

{'raw': '```json\n{\n  "caption": "Grupa zwolenników Donalda Trumpa zgromadziła się na wiecu, trzymając transparenty i plakaty z hasłami poparcia dla byłego prezydenta USA. Na twarzach uczestników widać entuzjazm i zaangażowanie polityczne.",\n  "tags": [\n    {"tag": "ludzie", "confidence": 0.75},\n    {"tag": "grupa", "confidence": 0.79},\n    {"tag": "transparent", "confidence": 0.78},\n    {"tag": "wydarzenie polityczne", "confidence": 0.8},\n    {"tag": "Donald Trump", "confidence": 0.9},\n    {"tag": "wiec", "confidence": 0.85},\n    {"tag": "mężczyzna", "confidence": 0.96},\n    {"tag": "kobieta", "confidence": 0.97},\n    {"tag": "uśmiech", "confidence": 0.94},\n    {"tag": "na zewnątrz", "confidence": 0.83}\n  ]\n}\n```'}

In [15]:
generate_caption_and_tags(sample_json2)

{'caption': 'Na torze wyścigowym kilku dżokejów rywalizuje w dynamicznym biegu konnym, a tłum obserwuje z trybun emocjonujące zmagania. W tle widać stadion i barwne stroje zawodników, podkreślające sportowy charakter wydarzenia.',
 'tags': [{'tag': 'wyścigi konne', 'confidence': 0.87},
  {'tag': 'dżokej', 'confidence': 0.87},
  {'tag': 'koń', 'confidence': 0.98},
  {'tag': 'stadion', 'confidence': 0.98},
  {'tag': 'sport', 'confidence': 0.86},
  {'tag': 'tor wyścigowy', 'confidence': 0.55},
  {'tag': 'na zewnątrz', 'confidence': 0.95}]}

In [16]:
def get_embedding(text: str):
    emb = client.embeddings.create(
        model=EMBEDDING_DEPLOYMENT,
        input=text
    )
    return emb.data[0].embedding

In [18]:
test_embedding = get_embedding('Na torze wyścigowym kilku dżokejów rywalizuje w dynamicznym biegu konnym')

In [19]:
len(test_embedding)

1536

In [20]:
embed_client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_version=AZURE_OPENAI_EMBEDDINGS_API_VERSION
)

In [21]:
def get_embedding2(text: str):
    emb = embed_client.embeddings.create(
        model=EMBEDDING_DEPLOYMENT,
        input=text
    )
    return emb.data[0].embedding

In [22]:
test_embedding2 = get_embedding2('Na torze wyścigowym kilku dżokejów rywalizuje w dynamicznym biegu konnym')

In [23]:
test_embedding == test_embedding2

True